# Soccer Data Scrapping

## IMPORTS

In [1]:
import numpy as np
import pandas as pd 
from requests import get
from bs4 import BeautifulSoup as soup
import re
from datetime import date
from tqdm.notebook import tqdm
import json

## INIT LEAGUE 1 DATA

In [1]:
import sys
from os import path
import csv
import requests
from bs4 import BeautifulSoup

In [8]:
BASE_URL = "https://www.soccerstats.com/"


def scrape_url_matches_by_month(league, month):
    
    #Request
    response = requests.get(
        BASE_URL + 'results.asp',
        params={'league': league, 'pmtype': month}
        )
    
    #Soup
    soup = BeautifulSoup(response.text, "html.parser")
    
    #Table of matches
    table = soup.find('table', id= 'btable')
    
    #Return list of href
    
    print([date.text for date in table.find_all('font', color = "green")])
    print([score.text for score in table.find_all('font', color = "gray")])
    return [matche['href'] for matche in table.find_all('a', class_= "vsmall")]

def scrape_url_matches_by_year(league, months):
    print(f"Searching {league}")
    
    urls = []
    for month in months:
        urls = urls + scrape_url_matches_by_month(league, month)
        
    return urls

league = 'france_2020'
# months = [f'month{i}' for i in range(1,13)]
months=['month8']


scrape_url_matches_by_year(league, months)

Searching france_2020
['Fri 9 Aug', 'Sat 10 Aug', 'Sat 10 Aug', 'Sat 10 Aug', 'Sat 10 Aug', 'Sat 10 Aug', 'Sat 10 Aug', 'Sun 11 Aug', 'Sun 11 Aug', 'Sun 11 Aug', 'Fri 16 Aug', 'Sat 17 Aug', 'Sat 17 Aug', 'Sat 17 Aug', 'Sat 17 Aug', 'Sat 17 Aug', 'Sat 17 Aug', 'Sun 18 Aug', 'Sun 18 Aug', 'Sun 18 Aug', 'Sat 24 Aug', 'Sat 24 Aug', 'Sat 24 Aug', 'Sat 24 Aug', 'Sun 25 Aug', 'Sun 25 Aug', 'Sun 25 Aug', 'Tue 27 Aug', 'Wed 28 Aug', 'Wed 28 Aug', 'Fri 30 Aug', 'Sat 31 Aug', 'Sat 31 Aug', 'Sat 31 Aug', 'Sat 31 Aug', 'Sat 31 Aug']
['(0-2)', '(0-0)', '(3-1)', '(1-0)', '(1-2)', '(0-1)', '(1-0)', '(1-0)', '(1-0)', '(1-0)', '(3-0)', '(0-0)', '(0-0)', '(0-1)', '(1-0)', '(1-2)', '(0-0)', '(0-1)', '(0-0)', '(1-1)', '(0-0)', '(2-0)', '(0-0)', '(0-1)', '(2-0)', '(0-1)', '(0-0)', '(1-0)', '(1-0)', '(0-1)', '(0-2)', '(1-0)', '(0-0)', '(0-0)', '(2-0)', '(0-0)']


['pmatch.asp?league=france_2020&stats=8-15-16-2019',
 'pmatch.asp?league=france_2020&stats=4-7-8-2019',
 'pmatch.asp?league=france_2020&stats=7-13-14-2019',
 'pmatch.asp?league=france_2020&stats=6-11-12-2019',
 'pmatch.asp?league=france_2020&stats=9-17-18-2019',
 'pmatch.asp?league=france_2020&stats=1-1-2-2019',
 'pmatch.asp?league=france_2020&stats=5-9-10-2019',
 'pmatch.asp?league=france_2020&stats=3-5-6-2019',
 'pmatch.asp?league=france_2020&stats=10-19-20-2019',
 'pmatch.asp?league=france_2020&stats=2-3-4-2019',
 'pmatch.asp?league=france_2020&stats=11-16-13-2019',
 'pmatch.asp?league=france_2020&stats=17-6-7-2019',
 'pmatch.asp?league=france_2020&stats=20-10-5-2019',
 'pmatch.asp?league=france_2020&stats=12-14-1-2019',
 'pmatch.asp?league=france_2020&stats=19-20-15-2019',
 'pmatch.asp?league=france_2020&stats=18-4-9-2019',
 'pmatch.asp?league=france_2020&stats=15-12-17-2019',
 'pmatch.asp?league=france_2020&stats=14-18-11-2019',
 'pmatch.asp?league=france_2020&stats=16-8-19-2019',

In [187]:
def scrape_data_from_url(dico_data, url):
    #Request
    response = requests.get(BASE_URL + url)

    #Soup
    soup = BeautifulSoup(response.text, "html.parser")
    
    #Separate columns teams
    datas = soup.find_all('div', class_= "six columns")
    tm1 = datas[2].find_all('table') #infos on team 1
    tm2 = datas[3].find_all('table') #infos on team 2

    #------team names
    #tm0 = average for the league
    dico_data['tm1_name'].append(tm1[0].find('h2').text)
    dico_data['tm2_name'].append(tm2[0].find('h2').text)

    #------Home and away records
    #TM1 -> home
    tm1_records = tm1[5].find_all('tr')
    dico_data['tm1_points_h'].append(tm1_records[2].find_all('td')[2].text.replace('%','')) #home points in %
    dico_data['tm1_PPGH_h'].append(tm1_records[4].find_all("td")[1].text) #  Points Per Game at Home
    dico_data['tm1_GS_h'].append(tm1_records[7].find_all('td')[2].text.replace('%','')) # goals score in %
    dico_data['tm1_GC_h'].append(tm1_records[8].find_all('td')[2].text.replace('%','')) # goals conceded in %
    dico_data['tm0_GS_h'].append(tm1_records[9].find_all('td')[2].text.replace('%',''))#  League average (% goals)
    #TM2 -> away
    tm2_records = tm2[5].find_all('tr')
    dico_data['tm2_points_a'].append(tm2_records[2].find_all('td')[4].text.replace('%','')) #home points in %
    dico_data['tm2_PPGA_a'].append(tm2_records[5].find_all("td")[1].text)  # Points Per Game away
    dico_data['tm2_GS_a'].append(tm2_records[7].find_all('td')[4].text.replace('%','')) # goals score in %
    dico_data['tm2_GC_a'].append(tm2_records[8].find_all('td')[4].text.replace('%','')) # goals conceded in %
    dico_data['tm0_GS_a'].append(tm2_records[9].find_all('td')[4].text.replace('%',''))#  League average (% goals)

    #------ stats against league average
    #tm1 tm2 tm0 Average 
    tm1_stats = tm1[6].find_all('tr')[1].find('div').find_all('div')[0]
    tm2_stats = tm2[6].find_all('tr')[1].find('div').find_all('div')[0]
    dico_data['tm0_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[1].text)
    dico_data['tm1_wins'].append(tm1_stats.find_all('tr')[4].find_all('font')[0].text.replace('%','')) # % Wins 
    dico_data['tm2_wins'].append(tm2_stats.find_all('tr')[4].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_wins'].append(tm1_stats.find_all('tr')[4].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_defeats'].append(tm1_stats.find_all('tr')[5].find_all('font')[0].text.replace('%','')) # % Defeats 
    dico_data['tm2_defeats'].append(tm2_stats.find_all('tr')[5].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_defeats'].append(tm1_stats.find_all('tr')[5].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_GSG'].append(tm1_stats.find_all('tr')[7].find_all('font')[0].text) # Goals scored per game
    dico_data['tm2_GSG'].append(tm2_stats.find_all('tr')[7].find_all('font')[0].text)
    dico_data['tm0_GSG'].append(tm1_stats.find_all('tr')[7].find_all('font')[1].text)
    dico_data['tm1_GCG'].append(tm1_stats.find_all('tr')[9].find_all('font')[0].text) # Goals conceded per game
    dico_data['tm2_GCG'].append(tm2_stats.find_all('tr')[9].find_all('font')[0].text)
    dico_data['tm0_GCG'].append(tm1_stats.find_all('tr')[9].find_all('font')[1].text)
    dico_data['tm1_D'].append(tm1_stats.find_all('tr')[11].find_all('font')[0].text.replace('%','')) # % Draws
    dico_data['tm2_D'].append(tm2_stats.find_all('tr')[11].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_D'].append(tm1_stats.find_all('tr')[11].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_TGG'].append(tm1_stats.find_all('tr')[13].find_all('font')[0].text) # Total goals per game
    dico_data['tm2_TGG'].append(tm2_stats.find_all('tr')[13].find_all('font')[0].text)
    dico_data['tm0_TGG'].append(tm1_stats.find_all('tr')[13].find_all('font')[1].text)
    dico_data['tm1_25'].append(tm1_stats.find_all('tr')[15].find_all('font')[0].text.replace('%','')) # % matches over 2.5 goals
    dico_data['tm2_25'].append(tm2_stats.find_all('tr')[15].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_25'].append(tm1_stats.find_all('tr')[15].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_35'].append(tm1_stats.find_all('tr')[17].find_all('font')[0].text.replace('%','')) # % matches over 3.5 goals
    dico_data['tm2_35'].append(tm2_stats.find_all('tr')[17].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_35'].append(tm1_stats.find_all('tr')[17].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_BTS'].append(tm1_stats.find_all('tr')[19].find_all('font')[0].text.replace('%','')) # % matches where both teams scored
    dico_data['tm2_BTS'].append(tm2_stats.find_all('tr')[19].find_all('font')[0].text.replace('%',''))
    dico_data['tm0_BTS'].append(tm1_stats.find_all('tr')[19].find_all('font')[1].text.replace('%',''))
    
    #tm1 and tm0 HOME
    tm1_stats = tm1[6].find_all('tr')[1].find('div').find_all('div')[1]
    # tm1_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[0].text # Points per game
    dico_data['tm0_PPGH_h'].append(tm1_stats.find_all('tr')[1].find_all('font')[1].text)
    dico_data['tm1_wins_h'].append(tm1_stats.find_all('tr')[4].find_all('font')[0].text.replace('%','')) # % Wins 
    dico_data['tm0_wins_h'].append(tm1_stats.find_all('tr')[4].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_defeats_h'].append(tm1_stats.find_all('tr')[5].find_all('font')[0].text.replace('%','')) # % Defeats 
    dico_data['tm0_defeats_h'].append(tm1_stats.find_all('tr')[5].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_GSG_h'].append(tm1_stats.find_all('tr')[7].find_all('font')[0].text) # Goals scored per game
    dico_data['tm0_GSG_h'].append(tm1_stats.find_all('tr')[7].find_all('font')[1].text)
    dico_data['tm1_GCG_h'].append(tm1_stats.find_all('tr')[9].find_all('font')[0].text) # Goals conceded per game
    dico_data['tm0_GCG_h'].append(tm1_stats.find_all('tr')[9].find_all('font')[1].text)
    dico_data['tm1_D_h'].append(tm1_stats.find_all('tr')[11].find_all('font')[0].text.replace('%','')) # % Draws
    dico_data['tm0_D_h'].append(tm1_stats.find_all('tr')[11].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_TGG_h'].append(tm1_stats.find_all('tr')[13].find_all('font')[0].text) # Total goals per game
    dico_data['tm0_TGG_h'].append(tm1_stats.find_all('tr')[13].find_all('font')[1].text)
    dico_data['tm1_25_h'].append(tm1_stats.find_all('tr')[15].find_all('font')[0].text.replace('%','')) # % matches over 2.5 goals
    dico_data['tm0_25_h'].append(tm1_stats.find_all('tr')[15].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_35_h'].append(tm1_stats.find_all('tr')[17].find_all('font')[0].text.replace('%','')) # % matches over 3.5 goals
    dico_data['tm0_35_h'].append(tm1_stats.find_all('tr')[17].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_BTS_h'].append(tm1_stats.find_all('tr')[19].find_all('font')[0].text.replace('%','')) # % matches where both teams scored
    dico_data['tm0_BTS_h'].append(tm1_stats.find_all('tr')[19].find_all('font')[1].text.replace('%',''))
    
    #tm2 and tm0 AWAY
    tm2_stats = tm2[6].find_all('tr')[1].find('div').find_all('div')[2]
    # tm1_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[0].text # Points per game
    dico_data['tm0_PPGH_a'].append(tm2_stats.find_all('tr')[1].find_all('font')[1].text)
    dico_data['tm1_wins_a'].append(tm2_stats.find_all('tr')[4].find_all('font')[0].text.replace('%','')) # % Wins 
    dico_data['tm0_wins_a'].append(tm2_stats.find_all('tr')[4].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_defeats_a'].append(tm2_stats.find_all('tr')[5].find_all('font')[0].text.replace('%','')) # % Defeats 
    dico_data['tm0_defeats_a'].append(tm2_stats.find_all('tr')[5].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_GSG_a'].append(tm2_stats.find_all('tr')[7].find_all('font')[0].text) # Goals scored per game
    dico_data['tm0_GSG_a'].append(tm2_stats.find_all('tr')[7].find_all('font')[1].text)
    dico_data['tm1_GCG_a'].append(tm2_stats.find_all('tr')[9].find_all('font')[0].text) # Goals conceded per game
    dico_data['tm0_GCG_a'].append(tm2_stats.find_all('tr')[9].find_all('font')[1].text)
    dico_data['tm1_D_a'].append(tm2_stats.find_all('tr')[11].find_all('font')[0].text.replace('%','')) # % Draws
    dico_data['tm0_D_a'].append(tm2_stats.find_all('tr')[11].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_TGG_a'].append(tm2_stats.find_all('tr')[13].find_all('font')[0].text) # Total goals per game
    dico_data['tm0_TGG_a'].append(tm2_stats.find_all('tr')[13].find_all('font')[1].text)
    dico_data['tm1_25_a'].append(tm2_stats.find_all('tr')[15].find_all('font')[0].text.replace('%','')) # % matches over 2.5 goals
    dico_data['tm0_25_a'].append(tm2_stats.find_all('tr')[15].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_35_a'].append(tm2_stats.find_all('tr')[17].find_all('font')[0].text.replace('%','')) # % matches over 3.5 goals
    dico_data['tm0_35_a'].append(tm2_stats.find_all('tr')[17].find_all('font')[1].text.replace('%',''))
    dico_data['tm1_BTS_a'].append(tm2_stats.find_all('tr')[19].find_all('font')[0].text.replace('%','')) # % matches where both teams scored
    dico_data['tm0_BTS_a'].append(tm2_stats.find_all('tr')[19].find_all('font')[1].text.replace('%',''))
    
    
def init_dico_data():
#Initialize Dico data
    dico_data = {}
    dico_keys = ['tm1_name', 'tm2_name', 'tm1_points_h', 'tm1_PPGH_h', 'tm1_GS_h', 'tm1_GC_h', 'tm0_GS_h',\
                 'tm2_points_a', 'tm2_PPGA_a', 'tm2_GS_a', 'tm2_GC_a', 'tm0_GS_a', 'tm0_PPGH', 'tm1_wins',\
                 'tm2_wins', 'tm0_wins', 'tm1_defeats', 'tm2_defeats', 'tm0_defeats', 'tm1_GSG', 'tm2_GSG',\
                 'tm0_GSG', 'tm1_GCG', 'tm2_GCG', 'tm0_GCG', 'tm1_D', 'tm2_D', 'tm0_D', 'tm1_TGG', 'tm2_TGG',\
                 'tm0_TGG', 'tm1_25', 'tm2_25', 'tm0_25', 'tm1_35', 'tm2_35', 'tm0_35', 'tm1_BTS', 'tm2_BTS',\
                 'tm0_BTS', 'tm0_PPGH_h', 'tm1_wins_h', 'tm0_wins_h', 'tm1_defeats_h', 'tm0_defeats_h',\
                 'tm1_GSG_h', 'tm0_GSG_h', 'tm1_GCG_h', 'tm0_GCG_h', 'tm1_D_h', 'tm0_D_h', 'tm1_TGG_h',\
                 'tm0_TGG_h', 'tm1_25_h', 'tm0_25_h', 'tm1_35_h', 'tm0_35_h', 'tm1_BTS_h', 'tm0_BTS_h',\
                 'tm0_PPGH_a', 'tm1_wins_a', 'tm0_wins_a', 'tm1_defeats_a', 'tm0_defeats_a', 'tm1_GSG_a',\
                 'tm0_GSG_a', 'tm1_GCG_a', 'tm0_GCG_a', 'tm1_D_a', 'tm0_D_a', 'tm1_TGG_a', 'tm0_TGG_a',\
                 'tm1_25_a', 'tm0_25_a', 'tm1_35_a', 'tm0_35_a', 'tm1_BTS_a', 'tm0_BTS_a']
    for k in dico_keys :
        dico_data[k] = []
    return dico_data

url='pmatch.asp?league=france_2020&stats=8-15-16-2019'  
dico_data = init_dico_data()
scrape_data_from_url(dico_data, url)
pd.DataFrame(dico_data)

,tm1_name,tm2_name,tm1_points_h,tm1_PPGH_h,tm1_GS_h,tm1_GC_h,tm0_GS_h,tm2_points_a,tm2_PPGA_a,tm2_GS_a,...,tm1_D_a,tm0_D_a,tm1_TGG_a,tm0_TGG_a,tm1_25_a,tm0_25_a,tm1_35_a,tm0_35_a,tm1_BTS_a,tm0_BTS_a
0,Monaco,Lyon,65,1.86,66,50,59,52,1.4,60,...,20,25,3.0,2.52,67,47,33,25,60,48


In [177]:
pd.DataFrame(dico_data)

,tm1_name,tm2_name,tm1_records,tm1_points,tm1_PPGH,tm1_GS,tm1_GC,tm1_LA,tm2_records,tm2_points,...,tm2_stats,tm2_wins,tm2_defeats,tm2_GSG,tm2_GCG,tm2_D,tm2_TGG,tm2_25,tm2_35,tm2_BTS
0,Monaco,Lyon,"[[[\n, <h2>Home and away records</h2>, \n]], [...",65,1.86,66,50,59,"[[[\n, <h2>Home and away records</h2>, \n]], [...",52,...,"[[], \n, [\n, [<td width=""52%""></td>, <td alig...",40,40,1.67,1.33,20,3.0,67,33,60


In [173]:
def init_dico_data():
#Initialize Dico data
    dico_data = {}
    dico_keys = ['tm1_name', 'tm2_name', 'tm1_records', 'tm1_points', 'tm1_PPGH', 'tm1_GS', 'tm1_GC', 'tm1_LA', \
                  'tm2_records', 'tm2_points', 'tm2_PPGA', 'tm2_GS', 'tm2_GC', 'tm2_LA', 'tm1_stats', 'tm0_PPGH', \
                  'tm1_wins', 'tm0_wins', 'tm1_defeats', 'tm0_defeats', 'tm1_GSG', 'tm0_GSG', 'tm1_GCG', 'tm0_GCG', \
                  'tm1_D', 'tm0_D', 'tm1_TGG', 'tm0_TGG', 'tm1_25', 'tm0_25', 'tm1_35', 'tm0_35', 'tm1_BTS', 'tm0_BTS',\
                  'tm2_stats', 'tm2_wins', 'tm2_defeats', 'tm2_GSG', 'tm2_GCG', 'tm2_D', 'tm2_TGG', 'tm2_25', 'tm2_35', 'tm2_BTS']
    for k in dico_keys :
        dico_data[k] = []
    return dico_data

dico_data = init_dico_data()
dico_data

{'tm1_name': [],
 'tm2_name': [],
 'tm1_records': [],
 'tm1_points': [],
 'tm1_PPGH': [],
 'tm1_GS': [],
 'tm1_GC': [],
 'tm1_LA': [],
 'tm2_records': [],
 'tm2_points': [],
 'tm2_PPGA': [],
 'tm2_GS': [],
 'tm2_GC': [],
 'tm2_LA': [],
 'tm1_stats': [],
 'tm0_PPGH': [],
 'tm1_wins': [],
 'tm0_wins': [],
 'tm1_defeats': [],
 'tm0_defeats': [],
 'tm1_GSG': [],
 'tm0_GSG': [],
 'tm1_GCG': [],
 'tm0_GCG': [],
 'tm1_D': [],
 'tm0_D': [],
 'tm1_TGG': [],
 'tm0_TGG': [],
 'tm1_25': [],
 'tm0_25': [],
 'tm1_35': [],
 'tm0_35': [],
 'tm1_BTS': [],
 'tm0_BTS': [],
 'tm2_stats': [],
 'tm2_wins': [],
 'tm2_defeats': [],
 'tm2_GSG': [],
 'tm2_GCG': [],
 'tm2_D': [],
 'tm2_TGG': [],
 'tm2_25': [],
 'tm2_35': [],
 'tm2_BTS': []}

In [160]:
datas = page.find_all('div', class_= "six columns")
tm1 = datas[2].find_all('table') #infos on team 1
tm2 = datas[3].find_all('table') #infos on team 2

#------team names
#tm0 = average for the league
dico_data['tm1_name'].append(tm1[0].find('h2').text)
dico_data['tm2_name'].append(tm2[0].find('h2').text)

#------Home and away records
#TM1 -> home
dico_data['tm1_records'].append(tm1[5].find_all('tr')
dico_data['tm1_points'].append(tm1_records[2].find_all('td')[2].text.replace('%','')) #home points in %
dico_data['tm1_PPGH'].append(tm1_records[4].find_all("td")[1].text) #  Points Per Game at Home
dico_data['tm1_GS'].append(tm1_records[7].find_all('td')[2].text.replace('%','')) # goals score in %
dico_data['tm1_GC'].append(tm1_records[8].find_all('td')[2].text.replace('%','')) # goals conceded in %
dico_data['tm1_LA'].append(tm1_records[9].find_all('td')[2].text.replace('%',''))#  League average (% goals)
#TM2 -> away
dico_data['tm2_records'].append(tm2[5].find_all('tr'))
dico_data['tm2_points'].append(tm2_records[2].find_all('td')[4].text.replace('%','')) #home points in %
dico_data['tm2_PPGA'].append(tm2_records[5].find_all("td")[1].text)  # Points Per Game away
dico_data['tm2_GS'].append(tm2_records[7].find_all('td')[4].text.replace('%','')) # goals score in %
dico_data['tm2_GC'].append(tm2_records[8].find_all('td')[4].text.replace('%','')) # goals conceded in %
dico_data['tm2_LA'].append(tm2_records[9].find_all('td')[4].text.replace('%',''))#  League average (% goals)

#------ stats against league average
#tm1 and tm0
dico_data['tm1_stats'].append(tm1[6].find_all('tr')[1].find('div').find_all('div')[1])
# tm1_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[0].text # Points per game
dico_data['tm0_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[1].text)
dico_data['tm1_wins'].append(tm1_stats.find_all('tr')[4].find_all('font')[0].text.replace('%','')) # % Wins 
dico_data['tm0_wins'].append(tm1_stats.find_all('tr')[4].find_all('font')[1].text.replace('%',''))
dico_data['tm1_defeats'].append(tm1_stats.find_all('tr')[5].find_all('font')[0].text.replace('%','')) # % Defeats 
dico_data['tm0_defeats'].append(tm1_stats.find_all('tr')[5].find_all('font')[1].text.replace('%',''))
dico_data['tm1_GSG'].append(tm1_stats.find_all('tr')[7].find_all('font')[0].text) # Goals scored per game
dico_data['tm0_GSG'].append(tm1_stats.find_all('tr')[7].find_all('font')[1].text)
dico_data['tm1_GCG'].append(tm1_stats.find_all('tr')[9].find_all('font')[0].text) # Goals conceded per game
dico_data['tm0_GCG'].append(tm1_stats.find_all('tr')[9].find_all('font')[1].text)
dico_data['tm1_D'].append(tm1_stats.find_all('tr')[11].find_all('font')[0].text.replace('%','')) # % Draws
dico_data['tm0_D'].append(tm1_stats.find_all('tr')[11].find_all('font')[1].text.replace('%',''))
dico_data['tm1_TGG'].append(tm1_stats.find_all('tr')[13].find_all('font')[0].text) # Total goals per game
dico_data['tm0_TGG'].append(tm1_stats.find_all('tr')[13].find_all('font')[1].text)
dico_data['tm1_25'].append(tm1_stats.find_all('tr')[15].find_all('font')[0].text.replace('%','')) # % matches over 2.5 goals
dico_data['tm0_25'].append(tm1_stats.find_all('tr')[15].find_all('font')[1].text.replace('%',''))
dico_data['tm1_35'].append(tm1_stats.find_all('tr')[17].find_all('font')[0].text.replace('%','')) # % matches over 3.5 goals
dico_data['tm0_35'].append(tm1_stats.find_all('tr')[17].find_all('font')[1].text.replace('%',''))
dico_data['tm1_BTS'].append(tm1_stats.find_all('tr')[19].find_all('font')[0].text.replace('%','')) # % matches where both teams scored
dico_data['tm0_BTS'].append(tm1_stats.find_all('tr')[19].find_all('font')[1].text.replace('%',''))
#tm2
dico_data['tm2_stats'].append(tm2[6].find_all('tr')[1].find('div').find_all('div')[2])
# tm1_PPGH'].append(tm1_stats.find_all('tr')[1].find_all('font')[0].text # Points per game
dico_data['tm2_wins'].append(tm2_stats.find_all('tr')[4].find_all('font')[0].text.replace('%','')) # % Wins 
dico_data['tm2_defeats'].append(tm2_stats.find_all('tr')[5].find_all('font')[0].text.replace('%','')) # % Defeats 
dico_data['tm2_GSG'].append(tm2_stats.find_all('tr')[7].find_all('font')[0].text) # Goals scored per game
dico_data['tm2_GCG'].append(tm2_stats.find_all('tr')[9].find_all('font')[0].text) # Goals conceded per game
dico_data['tm2_D'].append(tm2_stats.find_all('tr')[11].find_all('font')[0].text.replace('%','')) # % Draws
dico_data['tm2_TGG'].append(tm2_stats.find_all('tr')[13].find_all('font')[0].text # Total goals per game
dico_data['tm2_25'].append(tm2_stats.find_all('tr')[15].find_all('font')[0].text.replace('%','')) # % matches over 2.5 goals
dico_data['tm2_35'].append(tm2_stats.find_all('tr')[17].find_all('font')[0].text.replace('%','')) # % matches over 3.5 goals
dico_data['tm2_BTS'].append(tm2_stats.find_all('tr')[19].find_all('font')[0].text.replace('%','')) # % matches where both teams scored


In [185]:
datas = page.find_all('div', class_= "six columns")
tm1 = datas[2].find_all('table') #infos on team 1
tm2 = datas[3].find_all('table') #infos on team 2


tm2[6].find_all('tr')[1].find('div').find_all('div')[0]

<div class="tabbertab tabbertabdefault" title="
Total
"><br/>
<table border="0" cellpadding="3" cellspacing="0" width="100%">
<tr bgcolor="#999999" height="26"><td width="52%"></td><td align="center" valign="middle" width="25%"><font color="#fafafa"><b>Lyon</b></font></td>
<td align="center" valign="middle" width="23%"><font color="#fafafa">League average</font></td></tr>
</table>
<table border="0" cellpadding="2" cellspacing="0" style="margin-top:3px" width="100%">
<tr height="26"><td align="center" valign="middle" width="100%">
<table cellpadding="4" cellspacing="0" width="100%">
<tr>
<td align="right" bgcolor="#dedede" height="20" width="55%">
Points per game
 </td>
<td align="center" bgcolor="#D7EFBE" valign="middle" width="25%">
<font size="2"><b>1.43</b></font>
</td>
<td align="center" bgcolor="#D7EFBE" style="margin-bottom:4px;" valign="middle" width="20%"><font color="#404040">1.37</font></td>
</tr>
</table>
</td></tr>
<tr height="26"><td align="center" valign="middle" width="1

In [ ]:
https://www.soccerstats.com/results.asp?league=france

In [11]:
import re
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

#Request
BASE_URL = "https://www.soccerstats.com/"
league = 'france'
response = requests.get(
    BASE_URL + 'results.asp',
    params={'league': league}
    )

#Soup
soup = BeautifulSoup(response.text, "html.parser")

scores = soup.find_all(text=re.compile('h2h'))
divs = [score.parent.parent.parent for score in scores]
print(f'{len(divs)} matches trouvés ce mois ci !')

name = divs[0].text.replace('\xa0', ' ').replace('h2h', '')
url = divs[0].find_all('a', class_= "vsmall")[0]['href']


In [12]:
divs[0]

<tr class="odd" height="28"><td align="right" style="padding-right:5px;"><font color="green" size="1">Sun 16 May</font></td><td align="right">Bordeaux </td><td align="center"><font color="green" size="1">20:00</font></td><td align="left"> Lens</td><td align="center" valign="middle" width="45"><a class="vsmall" href="h2h.asp?league=france&amp;t1id=1&amp;t2id=14&amp;ly=2021">h2h</a></td><td align="center"><font color="gray" style="font-size:11px;"></font></td><td colspan="3"></td></tr>

In [18]:
divs[0].find_all('a', class_= "vsmall")[0]['href']

'h2h.asp?league=france&t1id=1&t2id=14&ly=2021'

In [23]:
' '.join([dt.text for dt in divs[0].find_all('font', color = "green")])

'Sun 16 May 20:00'

In [26]:
name = divs[0].text.replace('\xa0', ' ').replace('h2h', '')
url = divs[0].find_all('a', class_= "vsmall")[0]['href']

'Sun 16 MayBordeaux 20:00 Lens'

In [27]:
dico_matches = {
    'names' : [dvs.text.replace('\xa0', ' ').replace('h2h', '') for dvs in divs],
    'urls' : [dvs.find_all('a', class_= "vsmall")[0]['href'] for dvs in divs]
    }
df = pd.DataFrame(dico_matches)

In [30]:
df.iloc[0].urls

'h2h.asp?league=france&t1id=1&t2id=14&ly=2021'

In [39]:
df[df['names'] == 'Sun 16 MayBordeaux 20:00 Lens'].urls.values[0]

'h2h.asp?league=france&t1id=1&t2id=14&ly=2021'